#Analytics Vidhya - Big Mart Sales Prediction using XG Boost

---



In [1]:
# Importing the libraries

import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [2]:
pip install sklearn

In [46]:
# Importing the dataset

dataset = pd.read_csv('train_v9rqX0R.csv')

# dropping passed columns 
dataset.drop(["Item_Identifier", "Outlet_Identifier"], axis = 1, inplace = True)

X = dataset.iloc[:, :-1].values
y = dataset.iloc[:, -1].values
print(X[0:2,:])

[[9.3 'Low Fat' 0.016047301 'Dairy' 249.8092 1999 'Medium' 'Tier 1'
  'Supermarket Type1']
 [5.92 'Regular' 0.019278216 'Soft Drinks' 48.2692 2009 'Medium' 'Tier 3'
  'Supermarket Type2']]


In [47]:
# Taking care of missing data

from sklearn.impute import SimpleImputer

imputer = SimpleImputer(missing_values=np.nan, strategy='mean')
imputer.fit(X[:, 0:1])
X[:, 0:1] = imputer.transform(X[:, 0:1])

imputerC = SimpleImputer(missing_values=np.nan, strategy='most_frequent')
imputerC.fit(X[:, 6:7])
X[:, 6:7] = imputerC.transform(X[:, 6:7])
print(X[0:2,:])

[[9.3 'Low Fat' 0.016047301 'Dairy' 249.8092 1999 'Medium' 'Tier 1'
  'Supermarket Type1']
 [5.92 'Regular' 0.019278216 'Soft Drinks' 48.2692 2009 'Medium' 'Tier 3'
  'Supermarket Type2']]


In [48]:
# Encoding the Independent Variable

from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder

ct = ColumnTransformer(transformers=[('encoder', OneHotEncoder(), [1,6,7,8])], remainder='passthrough')
X = np.array(ct.fit_transform(X))
ct1 = ColumnTransformer(transformers=[('encoder', OneHotEncoder(), [17])], remainder='passthrough')
X = np.array(ct1.fit_transform(X))
print(X[0:2,:])

[[0.0 0.0 0.0 0.0 1.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 1.0
  0.0 0.0 0.0 0.0 1.0 0.0 1.0 0.0 0.0 0.0 1.0 0.0 0.0 9.3 0.016047301
  249.8092 1999]
 [0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 1.0 0.0 0.0 0.0
  1.0 0.0 0.0 0.0 1.0 0.0 0.0 0.0 1.0 0.0 0.0 1.0 0.0 5.92 0.019278216
  48.2692 2009]]


In [29]:
print(len(X[0,:]))

35


In [49]:
# Splitting the dataset into the Training set and Test set

from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 10)

In [50]:
# Feature Scaling

from sklearn.preprocessing import StandardScaler

sc = StandardScaler()
X_train[:,33:35] = sc.fit_transform(X_train[:,33:35])
X_test[:,33:35] = sc.transform(X_test[:,33:35])
print(X_train[0:2,:])
print(X_test[0:2,:])

[[0.0 0.0 0.0 0.0 1.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0
  1.0 0.0 0.0 0.0 1.0 0.0 0.0 0.0 1.0 0.0 0.0 1.0 0.0 16.0 0.065328932
  -0.9983682771732724 1.3355882534350187]
 [0.0 0.0 0.0 0.0 0.0 0.0 1.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 1.0
  0.0 0.0 0.0 0.0 1.0 0.0 0.0 1.0 0.0 0.0 1.0 0.0 0.0 19.7 0.053330653
  -1.3475058922808114 0.502595323513695]]
[[0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 1.0 0.0 0.0 1.0
  0.0 0.0 0.0 0.0 1.0 0.0 0.0 0.0 1.0 0.0 0.0 0.0 1.0 12.857645184135976
  0.097429924 0.5196700287614647 -1.5203875062952346]
 [0.0 0.0 1.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0
  1.0 0.0 0.0 0.0 0.0 1.0 1.0 0.0 0.0 0.0 1.0 0.0 0.0 7.895 0.094567181
  -0.5795980909412061 -0.09239962643010784]]


In [51]:
"""from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import LinearRegression
poly_reg = PolynomialFeatures(degree = 4)
X_poly = poly_reg.fit_transform(X_train)
regressor = LinearRegression()
regressor.fit(X_poly, y_train)"""

# Training XGBoost on the Training set

from xgboost import XGBRegressor

classifier = XGBRegressor()
classifier.fit(X_train, y_train)

[13:46:13] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=1, gamma=0,
             importance_type='gain', learning_rate=0.1, max_delta_step=0,
             max_depth=3, min_child_weight=1, missing=None, n_estimators=100,
             n_jobs=1, nthread=None, objective='reg:linear', random_state=0,
             reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
             silent=None, subsample=1, verbosity=1)

In [52]:
# Predicting the Test set results

y_pred = classifier.predict(X_test)

In [53]:
from sklearn.metrics import mean_squared_error
from math import sqrt

rms = sqrt(mean_squared_error(y_test, y_pred))
print(rms)

1086.9073396035722


In [35]:
print(rms)

1516.394533850651


# Applying on Sample Data for Submission

In [54]:
# Importing the Test dataset

dataset1 = pd.read_csv('test_AbJTz2l.csv')

dataset1.drop(["Item_Identifier", "Outlet_Identifier"], axis = 1, inplace = True)

X_test_o = dataset1.iloc[:, :].values
#print(X_test_o[1:5,:])

In [55]:
# Taking care of missing data

X_test_o[:, 0:1] = imputer.transform(X_test_o[:, 0:1])
X_test_o[:, 6:7] = imputerC.transform(X_test_o[:, 6:7])

#print(X_test_o[1:5,:])

In [56]:
# Encoding the Independent Variable

X_test_o = np.array(ct.fit_transform(X_test_o))
X_test_o = np.array(ct1.fit_transform(X_test_o))
#print(X_test_o[1,-5:-1])

In [57]:
X_test_o[:,33:35] = sc.transform(X_test_o[:,33:35])

In [58]:
# Predicting the Test set results

y_pred_o = classifier.predict(X_test_o)
#print(y_pred_o)

In [60]:
y_pred_o = pd.DataFrame(y_pred_o)
y_pred_o.to_csv('name_csv_file.csv')